In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import logging, sys, gc
import pandas as pd
from scipy.stats import binned_statistic
from scipy.optimize import curve_fit
from frontiers_analysis import load_tissue

In [ ]:
tissues = ["Brain", "Peripheral_Blood"]
colors = ["orange", "blue"]
tissue, other_tissue = tissues

In [ ]:
df = pd.read_csv(f"mca/mainTable_{tissue}.csv", index_col=0)
#df = pd.read_csv("../Smartseq3.HEK.fwdprimer.UMIcounts.txt", sep="\t")
M = df.sum(0)
f = df.divide(M,1).mean(1)
O = df.apply(lambda x: (x>0).sum(), 1)

In [ ]:
other_df = pd.read_csv(f"mca/mainTable_{other_tissue}.csv", index_col=0)
#other_df = pd.read_csv("../Smartseq3.Fibroblasts.NovaSeq.UMIcounts.txt", sep="\t")
other_M = other_df.sum(0)
other_f = other_df.divide(other_M,1).mean(1)
other_O = other_df.apply(lambda x: (x>0).sum(), 1)

In [ ]:
ortho = pd.read_csv("orthologues.txt").set_index("Gene stable ID")
other_df = other_df.join(ortho["Human gene stable ID"], how="inner").set_index("Human gene stable ID")

In [ ]:
cell_types = pd.read_csv("DGE/Adult-Brain_anno.csv", index_col=0).append(
pd.read_csv("DGE/Adult-Kidney_anno.csv", index_col=0)).append(
pd.read_csv("DGE/Adult-Muscle_anno.csv", index_col=0)).append(
pd.read_csv("DGE/Peripheral-Blood_anno.csv", index_col=0))

In [ ]:
merged_df = df.join(other_df, how="outer", lsuffix=tissues[0], rsuffix=tissues[1]).fillna(0)
print(merged_df.shape)
merged_M = merged_df.sum(0)
merged_f = merged_df.divide(merged_M,1).mean(1)
merged_O = merged_df.apply(lambda x: (x>0).sum(), 1)

In [ ]:
from methods import mazzolini as sampling

# Create models

In [ ]:
model = sampling(M=M, f=f)
other_model = sampling(M=other_M, f=other_f)
models = [model, other_model]
for method in models:
    print(method)
    method.run()
    
merged_model = sampling(M=merged_M, f=merged_f)
merged_model.run()

## Zipf

In [ ]:
plt.plot(np.sort(f/f.sum())[::-1], lw=10, c=colors[0], alpha=0.5, label=tissue)
plt.plot(np.sort(other_f/other_f.sum())[::-1], lw=10, c=colors[1], alpha=0.5, label=other_tissue)

for model, c in zip(models, colors):
    plt.plot(model.get_f(), lw=10, alpha=0.5, ls=":", c="dark"+c)

plt.xlabel("i")
plt.ylabel("f")

plt.xscale("log")
plt.yscale("log")

plt.legend()

## Heaps

In [ ]:
h = merged_df.apply(lambda x: (x>0).sum())

In [ ]:
bins = np.logspace(np.log10(merged_M.min()), np.log10(merged_M.max()), 35)
#bins = np.linspace(M.min(), M.max(), 35)

plt.scatter(merged_M, h, c="gray", alpha=0.8, label="data")

means, edges, _ = binned_statistic(merged_M, h, bins=bins)
var, edges, _ = binned_statistic(merged_M, h, statistic="std", bins=bins)
cnt, edges, _ = binned_statistic(merged_M, h, statistic="count", bins=bins)
var = var*var
mask = cnt > 10
means = means[mask]
var = var[mask]
l_edges = (edges[:-1])[mask]
r_edges = (edges[1:])[mask]
plt.hlines(means, l_edges, r_edges, lw=5, color="black", ls="--")

print(model.name_)
means, edges, _ = binned_statistic(merged_M, merged_model.get_h(), bins=bins)
var, edges, _ = binned_statistic(merged_M, merged_model.get_h(), statistic="std", bins=bins)
cnt, edges, _ = binned_statistic(merged_M, merged_model.get_h(), statistic="count", bins=bins)
var = var*var
mask = cnt > 100
means = means[mask]
var = var[mask]
l_edges = (edges[:-1])[mask]
r_edges = (edges[1:])[mask]

merged_model.hmean = means
merged_model.hvar = var
merged_model.cnt = cnt

#plt.scatter(merged_M, merged_model.get_h(), alpha=0.2, c=model.color_, label=model.name_)

plt.scatter(df.sum(0), df.apply(lambda x: (x>0).sum()), c=colors[0], marker="x", alpha=0.8, label=tissues[0])
plt.scatter(other_df.sum(0), other_df.apply(lambda x: (x>0).sum()), c=colors[1], marker="x", alpha=0.8, label=tissues[1])

plt.xscale("log")
plt.yscale("log")

plt.xlabel("M")
plt.ylabel("h")

plt.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(18,15))

masks = [(merged_df.columns.isin(subdf.columns)) for subdf in [df, other_df]]

ytext = 0.005

for mask, label, color in zip(masks, tissues, colors):
    data = (h - merged_model.get_h())[mask]
    ax.hist(data, histtype="step", lw=10, color=color, label=label, density=True)
    med = np.average(data)
    ax.vlines(med, 0, 0.006, lw=12, ls=":", color=color)
    ax.annotate("{mean:.0f}±{error:.0f}".format(mean=med, error=np.std(data)), (med, ytext), fontsize=30)
    ytext-=0.001
    
ax.set_xlabel("h-h_sampling", fontsize=35)
ax.set_ylabel("pdf", fontsize=35)
ax.tick_params(labelsize=30)
ax.legend(fontsize=30)

plt.show()

### Fluctuations

In [ ]:
h = merged_df.apply(lambda x: (x>0).sum())
means, edges, _ = binned_statistic(merged_M, h, bins=bins)
var, edges, _ = binned_statistic(merged_M, h, statistic="std", bins=bins)
var = var*var
mask = cnt > 100
means = means[mask]
var = var[mask]

x = means

plt.scatter(means, var, c="gray", alpha=0.8, label="data")

popt, pcov= curve_fit(lambda x, C: C*x, means, var)
plt.plot(x, popt[0]*x, lw=5, ls="--", c="cyan", alpha=0.8, label="C*<h>")

popt, pcov= curve_fit(lambda x, C: C*x*x, means, var)
plt.plot(x, popt[0]*x**2, lw=5, ls="--", c="purple", alpha=0.8, label ="C*<h>^2")

plt.xlabel("<h>")
plt.ylabel("var(h)")

plt.xscale("log")
plt.yscale("log")

plt.legend()

#plt.ylim(1e2,1e3)

## Predicted occurrences

In [ ]:
Os = []
for i in range(5):
    method = sampling(M=M, f=f)
    method.run()
    print(i, method)
    Os.append(method.get_O())
O_sampling = np.average(Os, axis=0)

Os = []
for i in range(5):
    method = sampling(M=other_M, f=other_f)
    method.run()
    print(i, method)
    Os.append(method.get_O())
O_other_sampling = np.average(Os, axis=0)

Os = []
for i in range(5):
    method = sampling(M=merged_M, f=merged_f)
    method.run()
    print(i, method)
    Os.append(method.get_O())
O_merged_sampling = np.average(Os, axis=0)

In [ ]:
mask = (O_merged_sampling-merged_O/merged_df.shape[1])>0.2

In [ ]:
fig, axs = plt.subplots(1, 1+len(models), figsize=(30, 15))

models[0].color_="gray"
models[1].color_="gray"
for model, ax in zip(models, axs):
    ax.set_title(model.name_, fontsize=35)
    
axs[0].scatter(O/df.shape[1], O_sampling, alpha=0.5, s=350, color="gray", marker="o")    
axs[1].scatter(other_O/other_df.shape[1], O_other_sampling, alpha=0.5, s=350, color="gray", marker="o")  
axs[2].scatter(merged_O/merged_df.shape[1], O_merged_sampling, alpha=0.5, s=350, color="gray", marker="o")
axs[2].scatter((merged_O/merged_df.shape[1])[mask], O_merged_sampling[mask], alpha=0.5, s=350, color="red", marker="o")

for ax in axs:
    ax.plot([0,1], [0,1], lw=20, alpha=0.7,ls="--", c="black")
    ax.tick_params(labelsize=40, width=5, size=10)
    ax.set_xlabel("$o_i$, empirical", fontsize=65)
    ax.set_ylabel("$o_i$, predicted", fontsize=65)
    
axs[0].set_title("Sampling model {}".format(tissue), fontsize=55)
axs[1].set_title("Sampling model {}".format(other_tissue), fontsize=55)
axs[2].set_title("Merged", fontsize=55)

plt.tight_layout()
plt.show()
fig.savefig(f"Oreal_Opred_poissonModel_{tissue}.pdf")

In [ ]:
for g in (merged_O/merged_df.shape[1])[mask].index:
    print(g)